In [1]:
import yaml
import pandas as pd
from message_ix.utils import make_df
from message_ix.models import MESSAGE_ITEMS
from collections.abc import Mapping
from itertools import repeat

<IPython.core.display.Javascript object>

In [2]:
with open('tech_data_01_update.yaml','r') as stream:
    tech_data = yaml.safe_load(stream)
    
years = [*range(700,730,10)]
regions = ['Westeros', 'Westerlands']

In [3]:
# scenario.regions is the list of nodes in the scenario
default_cost_by_region = {reg: 1 for reg in regions}
cost_by_region = {
    tech: tech_data['model_data']
            .get(tech,{})
            .get('cost_by_region', default_cost_by_region)
    for tech in set(tech_data) - set(['model_data'])
}

In [4]:
# scenario.years is the list of years in the scenario
first_active_year = tech_data['model_data'].get('first_active_year')
default_costreduction_by_region = {'rate': 0}
costreduction_by_region = {
    tech: tech_data['model_data']
            .get(tech,{})
            .get('costreduction_by_region', default_costreduction_by_region)
    for tech in set(tech_data) - set(['model_data'])
}

vtg_years = [y for y in years if y >= first_active_year]


In [5]:
years_dict = {yv:[ya for ya in years if ya >= yv] for yv in years if yv >= first_active_year}

vtg = [yv for ya in list(years_dict.keys()) for yv in repeat(ya,len(years_dict[ya]))]

act = [yv for ya in list(years_dict.keys()) for yv in repeat(ya,len(years_dict[ya]))]

years_vtg_act = pd.DataFrame(
    {
    'year_vtg':[k for k, v in years_dict.items() for i in range(len(v))],
    'year_act':[ya for value in years_dict.values() for ya in value]
    }
)

years_vtg_act

,year_vtg,year_act
0,700,700
1,700,710
2,700,720
3,710,710
4,710,720
5,720,720


In [6]:
costperiodfactor_by_region = {
    tech: {
        y: (1-costreduction_by_region.get(tech).get('rate'))**(y-first_active_year)
        for y in years_vtg_act['year_vtg']
    }
    for tech in set(tech_data)-set(['model_data'])
}

In [7]:
#par_dim = {'inv_cost':['node_loc'],'fix_cost':['node_loc'],'var_cost':['node_loc']}
parameters = {}
for tech in set(tech_data) - set(['model_data']):
    parameters.update({par: list(MESSAGE_ITEMS[par]['idx_names']) for par in set(tech_data[tech])})
data = {par: [] for par in list(parameters.keys())}

In [13]:
print(MESSAGE_ITEMS['technical_lifetime']['idx_names'],'-- technical_lifetime')
print(MESSAGE_ITEMS['growth_new_capacity_up']['idx_names'],'-- growth_new_capacity_up')
print(MESSAGE_ITEMS['initial_new_capacity_up']['idx_names'],'-- initial_new_capacity_up')
print(MESSAGE_ITEMS['inv_cost']['idx_names'],'-- inv_cost')
print('')
print(MESSAGE_ITEMS['capacity_factor']['idx_names'],'-- capacity_factor')
print('')
print(MESSAGE_ITEMS['var_cost']['idx_names'],'-- var_cost')
print(MESSAGE_ITEMS['emission_factor']['idx_names'],'-- emission_factor')
print('')
print(MESSAGE_ITEMS['input']['idx_names'],'-- input')


('node_loc', 'technology', 'year_vtg') -- technical_lifetime
('node_loc', 'technology', 'year_vtg') -- growth_new_capacity_up
('node_loc', 'technology', 'year_vtg') -- initial_new_capacity_up
('node_loc', 'technology', 'year_vtg') -- inv_cost

('node_loc', 'technology', 'year_vtg', 'year_act', 'time') -- capacity_factor

('node_loc', 'technology', 'year_vtg', 'year_act', 'mode', 'time') -- var_cost
('node_loc', 'technology', 'year_vtg', 'year_act', 'mode', 'emission') -- emission_factor

('node_loc', 'technology', 'year_vtg', 'year_act', 'mode', 'node_origin', 'commodity', 'level', 'time', 'time_origin') -- input


In [9]:
# Basic DataFrame
for tech, par_dict in tech_data.items():
    if tech != 'model_data':
        for par, par_data in par_dict.items():
            if not isinstance(par_data, Mapping):
                par_data = {'value': par_data, 'unit': '-'}
            data[par].append(
                make_df(
                    par,
                    technology=tech,
                    value=par_data['value'],
                    unit=par_data['unit'],
                )
            )

data = {k: pd.concat(v) for k, v in data.items()}

In [10]:
# Expanded DataFrame
data_expand ={par: [] for par in list(parameters.keys())} 
for par in list(parameters.keys()):
    if par in ['inv_cost','technical_lifetime','growth_new_capacity_up','initial_new_capacity_up']:
        for tech, diffs in tech_data['model_data'].items():
            if tech != 'first_active_year':
                for reg in regions:
                    for y in sorted(list(set(years_vtg_act['year_vtg']))):
                        if par == 'inv_cost':
                            value = (data[par]['value']
                                       *diffs.get('cost_by_region').get(reg)
                                       *costperiodfactor_by_region.get(tech).get(y)
                                      )
                        else:
                            value = data[par]['value']
                        data_expand[par].append(
                            data[par].assign(
                                node_loc=reg,
                                year_vtg=y,
                                value=value
                                ))


In [11]:
data_expand = {k: pd.concat(v) for k, v in data_expand.items() 
               if k in ['inv_cost','technical_lifetime','growth_new_capacity_up','initial_new_capacity_up']}

In [12]:
with pd.ExcelWriter('tech_data.xlsx', engine='xlsxwriter', mode='w') as writer:
    for sheet_name, sheet_data in data_expand.items():
        sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)